In [13]:
import subprocess
from io import BytesIO
from PIL import Image, ImageChops
import  cv2
import numpy as np
import schedule
import time
from datetime import datetime, timedelta
from wxpusher import WxPusher
import pytesseract
from skimage import io, color, metrics
import requests
import json
#截取当前屏幕
def take_screenshot(filename):
    # 截取当前虚拟机屏幕并传输到计算机上
    command = f"adb -s {全局_EMULATOR_NAME} exec-out screencap -p"
    screenshot_data = subprocess.check_output(command, shell=True)

    # 处理截图数据
    screenshot = Image.open(BytesIO(screenshot_data))
    # screenshot = screenshot.transpose(Image.Transpose.ROTATE_270)
    # 保存截图为指定的文件
    screenshot.save(filename)
    screenshot.close()

# 读取图像，解决imread不能读取中文路径的问题
def cv_imread(filepath):
    cv_img = cv2.imdecode(np.fromfile(filepath, dtype=np.uint8), -1)
    return cv_img

#计算结构相似性指数(SSIM)得到相似程度
def compare_images(image1_path, image2_path):
    img1 = io.imread(image1_path, as_gray=True)
    img2 = io.imread(image2_path, as_gray=True)

    # 校验图片尺寸是否相同
    if img1.shape != img2.shape:
        raise ValueError("两张图片的尺寸必须相同")

    # 计算结构相似性指数(SSIM)
    ssim_value = metrics.structural_similarity(img1, img2)

    return ssim_value
#通过wxpusher发送消息
def wxPusher_send_messaget_post(summary,content,topic_id,token):
    payload = {
    "appToken":token,
    "summary":summary,
    "content":content,
    "contentType":1,
    "topicIds":[ 
        topic_id
    ],
    "url":"https://wxpusher.zjiecode.com", 
}
    url= "https://wxpusher.zjiecode.com/api/send/message"

    request = requests.post(url, json=payload).json()
    
    return request


#对x，y坐标点击
def tap_screen(x, y):
    # 使用adb命令模拟屏幕点击
    command = f"adb -s {全局_EMULATOR_NAME} shell input tap {x} {y}"
    subprocess.run(command, shell=True)

#划动
def swipe_screen(start_x, start_y, end_x, end_y,duration_ms=300):
    
    command = f"adb -s {全局_EMULATOR_NAME} shell input swipe {start_x} {start_y} {end_x} {end_y} {duration_ms}"
    subprocess.run(command, shell=True)
 
#输入
def input_text_to_vm(text):
    # 使用adb shell input text命令将文本输入到指定虚拟机
    command = f'adb -s {全局_EMULATOR_NAME} shell input text "{text}"'
    subprocess.run(command, shell=True)


def recognize_number_in_image(image_path,start_x,start_y,end_x,end_y):
    # 打开图片
    image = Image.open(image_path)
    
    # 使用Tesseract OCR进行识别
    cropped_image = image.crop((start_x, start_y, end_x, end_y))
    # cropped_image.show()
    # result = pytesseract.image_to_string(image[start_x:end_x,start_y,end_y,:], config='--psm 6 outputbase digits')
    result = pytesseract.image_to_string(cropped_image, config='chi_tra_vert')
    
    return result

def find_image_in_larger(image_small_path,image_large_path):
    # 读取大图和小图
    image_large = cv_imread(image_large_path)
    image_small = cv_imread(image_small_path)

    # 检查图片是否读取成功
    if image_large is None or image_small is None:
        print("无法读取图片.")
        return

    # 使用模板匹配方法
    result = cv2.matchTemplate(image_large, image_small, cv2.TM_CCOEFF_NORMED)
    #zhao
    (minVal, maxVal, minLoc, maxLoc) = cv2.minMaxLoc(result)
    

    (startX, startY) = maxLoc
    center_x = startX + image_small.shape[1]/2
    center_y = startY + image_small.shape[0]/2
    return maxVal,center_x, center_y

In [14]:
print(pytesseract.get_languages(config=''))

['chi_sim', 'chi_sim_vert', 'chi_tra', 'chi_tra_vert', 'eng', 'osd']


In [15]:
a=recognize_number_in_image('图片/1.png',66,441,178,458)#我的血条
b=recognize_number_in_image('图片/1.png',315,441,472,458)#敌方血条
c=recognize_number_in_image('图片/1.png',81,458,180,473)#我方蓝条
d=recognize_number_in_image('图片/1.png',362,458,461,473)#敌方蓝条
a,b,c,d

('', 'Vig 11492/1.\n\n', '0/3070\n', '710/1710\n')

In [ ]:
def start():
    shot_path='图片/截图.png'
    tap_screen(66,811)
    time.sleep(1)
    
    tap_screen(253,500)#开始匹配
    while 1:
        take_screenshot(shot_path)
        val,_,_=find_image_in_larger('图片/匹配中.png',shot_path)
        if val<0.9:#匹配到人了
            break
        time.sleep(2)
    tap_screen(269,596)#进入战斗
    time.sleep(1)
    while 1:#开始战斗
        take_screenshot(shot_path)
        val,_,_=find_image_in_larger('图片/正在战斗.png',shot_path)
        if val<0.8:#匹配到人了
            break

        tap_screen(289,844)#点击绝招
        time.sleep(0.5)
        take_screenshot(shot_path)
        val,x,y=find_image_in_larger('图片/血魔斩.png',shot_path)
        time.sleep(0.5)
        if val>0.9:
            tap_screen(x,y)#使用血魔斩
    




        
